In [57]:
from torchmin import least_squares

In [2]:
import torch

## Simple Example
Third example from https://hernandis.me/2020/04/05/three-examples-of-nonlinear-least-squares-fitting-in-python-with-scipy.html

In [3]:
def h(theta, x, y):
    return theta[2] * (x - theta[0])**2 + theta[3] * (y - theta[1])**2

In [4]:
xs = torch.linspace(-1, 1, 20)

In [5]:
ys = torch.linspace(-1, 1, 20)

In [6]:
gridx, gridy = torch.meshgrid(xs, ys)

/home/junchi/sp1/myenv/lib/python3.7/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
x0 = 0.1; y0 = -0.15; a = 1; b = 2; noise = 0.1
hs = h([x0, y0, a, b], gridx, gridy)

In [9]:
hs += noise * torch.rand(hs.shape)

In [10]:
def fun(theta):
    return (h(theta, gridx, gridy) - hs).flatten()

In [15]:
theta0 = torch.tensor([1.0,2.0,3.0,4.0])
print(theta0.shape)

torch.Size([4])


In [16]:
least_squares(fun, theta0)

 active_mask: tensor([0., 0., 0., 0.])
        cost: tensor(0.2703)
         fun: tensor([-1.7471e-03, -4.9572e-03,  6.7794e-02,  1.5046e-02,  8.8193e-03,
         4.8228e-02,  4.8441e-02,  3.4462e-02, -9.0724e-03, -4.0262e-02,
        -4.5632e-03,  3.6764e-02,  3.5071e-02, -3.2262e-02,  2.3600e-02,
         6.5706e-03,  4.2486e-02,  1.6864e-02,  6.0347e-02,  1.0507e-01,
         6.3286e-03,  2.8369e-02,  2.6200e-02,  1.5559e-02, -3.3913e-02,
        -8.2383e-03, -5.9159e-03, -2.6908e-02, -1.6461e-02, -2.9558e-02,
        -4.8771e-02,  4.3956e-02, -4.5494e-02,  1.7209e-03, -6.6527e-03,
         3.5132e-02, -1.3797e-02,  5.4305e-02, -1.0414e-02,  3.1912e-03,
         3.9157e-02,  1.0020e-03,  4.3786e-02, -2.9698e-02,  4.4659e-02,
        -5.6294e-02,  2.7010e-02,  2.5887e-02, -3.7729e-02, -2.3957e-02,
         2.4721e-02,  2.4408e-03, -2.1249e-02,  2.5176e-02, -1.4634e-02,
         1.1047e-02,  2.2877e-02,  1.8584e-02,  4.0228e-02,  5.5419e-02,
        -4.6577e-03,  4.4930e-02,  8.5616e

## Homography Estimation

In [40]:
h_gt = torch.tensor([[1,0,10],
                     [0,1,5],
                     [0,0,1]]).float()

In [41]:
# sample points
xs = torch.linspace(0, 40-1, 40)
ys = torch.linspace(0, 30-1, 30)
gridx, gridy = torch.meshgrid(xs, ys)

/home/junchi/sp1/myenv/lib/python3.7/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [42]:
src_pts = torch.stack([gridx, gridy])
print(src_pts.shape)
src_pts[:,0,5]

torch.Size([2, 40, 30])


tensor([0., 5.])

In [43]:
src_pts = torch.concat((src_pts, torch.ones(src_pts.shape[1:])[None, :, :]), dim=0)

In [44]:
print(src_pts.shape)
src_pts[:,0,5]

torch.Size([3, 40, 30])


tensor([0., 5., 1.])

In [45]:
def apply_homography(H, pts_src):
    pts_dst = H @ pts_src
    pts_dst = pts_dst / pts_dst[-1, :]
    return pts_dst

In [46]:
src_pts = src_pts.reshape(3, -1)
dst_pts =  apply_homography(h_gt, src_pts)
print(dst_pts.shape)
dst_pts[:,10]

torch.Size([3, 1200])


tensor([10., 15.,  1.])

In [47]:
noise = 0.1
dst_pts += noise * torch.rand(dst_pts.shape)
dst_pts[:,10]

tensor([10.0974, 15.0206,  1.1000])

In [48]:
def fun(theta):
    theta = torch.cat([theta, torch.ones(1)])
    r = (apply_homography(theta.reshape(3,3), src_pts) - dst_pts)
    return r.flatten()

In [49]:
theta = torch.tensor([1,0,10,0,1,5,0,0])
theta = torch.cat([theta, torch.ones(1)])
print(theta)
print(theta.reshape(3,3))

tensor([ 1.,  0., 10.,  0.,  1.,  5.,  0.,  0.,  1.])
tensor([[ 1.,  0., 10.],
        [ 0.,  1.,  5.],
        [ 0.,  0.,  1.]])


In [50]:
theta0 = torch.eye(3).flatten()[:-1]
# theta0 = torch.tensor([1,0,11.0,0,1,6,0,0]).float()
theta0

tensor([1., 0., 0., 0., 1., 0., 0., 0.])

In [51]:
theta = torch.tensor([1,0,10,0,1,5,0,0])
r = fun(theta)
print(r.shape)

torch.Size([3600])


In [52]:
dst_pts = dst_pts
src_pts = src_pts
theta0 = theta0

In [53]:
r = fun(theta0)
r.device

device(type='cpu')

In [59]:
import torchimize
from torchimize.functions import lsq_gna
from torchimize.functions import lsq_lma

coeffs_list = lsq_lma(theta0, fun)

In [60]:
coeffs_list

[tensor([1., 0., 0., 0., 1., 0., 0., 0.]),
 tensor([ 1.0505,  0.0693,  0.0062,  0.0255,  1.0041,  0.0024, -0.0052, -0.0052]),
 tensor([ 1.1186,  0.1448,  0.0144,  0.0686,  1.0347,  0.0066, -0.0024, -0.0025]),
 tensor([ 1.2161e+00,  2.3459e-01,  2.7815e-02,  1.1833e-01,  1.0990e+00,
          1.3860e-02,  1.8550e-05, -1.0914e-04]),
 tensor([1.3651, 0.3355, 0.0529, 0.1628, 1.2334, 0.0271, 0.0029, 0.0036]),
 tensor([1.5401, 0.4301, 0.0972, 0.1872, 1.4227, 0.0471, 0.0060, 0.0083]),
 tensor([1.6840, 0.5072, 0.1798, 0.1964, 1.5979, 0.0741, 0.0084, 0.0127]),
 tensor([1.7533, 0.5467, 0.3623, 0.1983, 1.6928, 0.1177, 0.0097, 0.0151]),
 tensor([1.7455, 0.5355, 0.8327, 0.1944, 1.6965, 0.2277, 0.0097, 0.0152]),
 tensor([1.7455, 0.5355, 0.8327, 0.1944, 1.6965, 0.2277, 0.0097, 0.0152]),
 tensor([1.7430, 0.5367, 0.8217, 0.1907, 1.6931, 0.3394, 0.0097, 0.0152]),
 tensor([1.7430, 0.5367, 0.8217, 0.1907, 1.6931, 0.3394, 0.0097, 0.0152]),
 tensor([1.7092, 0.5089, 1.2936, 0.1850, 1.6637, 0.4497, 0.0093, 0.

In [61]:
%%time
least_squares(fun, theta0)

CPU times: user 21.6 s, sys: 0 ns, total: 21.6 s
Wall time: 2.76 s


 active_mask: tensor([0., 0., 0., 0., 0., 0., 0., 0.])
        cost: tensor(2.9713)
         fun: tensor([-0.0451,  0.0429,  0.0248,  ..., -0.0292, -0.0981, -0.0606])
        grad: tensor([-0.0033, -0.0013, -0.1134, -0.0134,  0.0120, -0.0833,  1.1609,  0.8150])
         jac: <torchmin.lstsq.linear_operator.TorchLinearOperator object at 0x7f934d9f3ed0>
     message: '`xtol` termination condition is satisfied.'
        nfev: 440
        njev: 434
  optimality: tensor(1.1609)
      status: 3
     success: True
           x: tensor([ 1.0001e+00, -4.3697e-04,  1.0053e+01,  2.7896e-05,  9.9963e-01,
         5.0505e+00,  4.9899e-06, -1.3778e-05])

In [108]:
print(type(res))
res

<class 'scipy.optimize.optimize.OptimizeResult'>


 active_mask: tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
        cost: tensor(2.9045, device='cuda:0')
         fun: tensor([-0.0025,  0.0137,  0.0036,  ..., -0.0176, -0.0534, -0.0486],
       device='cuda:0')
        grad: tensor([ 0.0434,  0.0162, -0.1346,  0.0444,  0.0527, -0.0974, -1.3460, -0.8048],
       device='cuda:0')
         jac: <torchmin.lstsq.linear_operator.TorchLinearOperator object at 0x7ff7182ff8d0>
     message: '`xtol` termination condition is satisfied.'
        nfev: 705
        njev: 698
  optimality: tensor(1.3460, device='cuda:0')
      status: 3
     success: True
           x: tensor([ 1.0004e+00, -5.3274e-05,  1.0048e+01,  2.1995e-05,  1.0003e+00,
         5.0490e+00,  4.7323e-06,  2.5037e-06], device='cuda:0')

In [103]:
res.x

tensor([ 1.0004e+00, -5.3274e-05,  1.0048e+01,  2.1995e-05,  1.0003e+00,
         5.0490e+00,  4.7323e-06,  2.5037e-06], device='cuda:0')

In [96]:
res.x.device

device(type='cpu')

In [33]:
def compute_homography(reference_points, observed_points):
    # Initialize the homography parameters to some initial values
    h = torch.tensor([[1.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0],
                      [0.0, 0.0, 1.0]], dtype=torch.float)

    h = torch.autograd.Variable(h).requires_grad_()

    # Define a function that computes the residuals
    def residuals(h):
        # Transform the reference points using the homography
        transformed_points = torch.matmul(h, reference_points)

        # Divide the transformed points by their third coordinate to normalize them
        transformed_points = transformed_points / transformed_points[2,:]

        # Compute the residuals as the difference between the observed points
        # and the transformed points
        residuals = observed_points - transformed_points

        # Enable gradient tracking for the residuals tensor
        residuals.requires_grad_()

        mse = torch.mean(residuals ** 2)

        # Return the MSE as a scalar tensor
        return mse

    # Define a function that computes the Jacobian matrix
    def jacobian(h):
        # Compute the Jacobian matrix of the residuals with respect to the homography parameters
        # This can be computed analytically, but for simplicity we will approximate it
        # by finite differences using the torch.autograd module
        with torch.enable_grad():
            h_grad = torch.autograd.grad(residuals(h), h, create_graph=True)

        # Reshape the tensor of gradients into a 2D tensor
        jacobian = torch.reshape(torch.stack(h_grad), (3, 3))

        # Return the Jacobian matrix as a tensor
        return jacobian

    # Define a stopping criterion (e.g., maximum number of iterations or convergence tolerance)
    max_iter = 100
    tol = 1e-6

    # Iterate until the stopping criterion is met
    for i in range(max_iter):
        # Compute the residuals and the Jacobian matrix
        r = residuals(h)
        j = jacobian(h)

        # Solve the linear system of equations using the Gaussian elimination method
        # This can be done using the torch.solve() function
        h_update, _ = torch.linalg.solve(r, j)

        # Update the homography parameters
        h = h + h_update

        # Check the stopping criterion
        if torch.norm(h_update) < tol:
            break

    # Return the estimated homography matrix
    return h

In [34]:
# Define some test points in the reference coordinate system
reference_points = torch.tensor([[1.0, 1.0, 1.0],
                                 [1.0, 2.0, 1.0],
                                 [2.0, 1.0, 1.0],
                                 [2.0, 2.0, 1.0]], dtype=torch.float, requires_grad=True)

# Define the corresponding points in the observed coordinate system
observed_points = torch.tensor([[2.0, 1.0, 1.0],
                                [2.0, 3.0, 1.0],
                                [3.0, 1.0, 1.0],
                                [3.0, 3.0, 1.0]], dtype=torch.float)

# Compute the homography matrix using the compute_homography() function
h = compute_homography(reference_points.T, observed_points.T)

# Print the resulting homography matrix
print(h)

# The expected output is:
# tensor([[1., 0., 1.],
#         [0., 1., 1.],
#         [0., 0., 1.]])

RuntimeError: linalg.solve: The input tensor must have at least 2 dimensions.

In [35]:
import numpy as np
def gauss_newton(X, Y, max_iters=100):
  # Initialize the homography matrix H to the identity matrix
  H = np.eye(3)

  # Repeat the Gauss-Newton algorithm for a maximum number of iterations
  for _ in range(max_iters):
    # Compute the error between the points and the transformed points
    error = Y - np.dot(X, H.T)

    # Compute the Jacobian matrix
    J = np.zeros((len(X), 8))
    for i in range(len(X)):
      J[i] = [-X[i, 0], -X[i, 1], -1, 0, 0, 0, X[i, 0]*error[i, 0], X[i, 1]*error[i, 0]]
      J[i] = [0, 0, 0, -X[i, 0], -X[i, 1], -1, X[i, 0]*error[i, 1], X[i, 1]*error[i, 1]]

    # Compute the Hessian matrix and the gradient vector
    H = np.dot(J.T, J)
    g = np.dot(J.T, error.flatten())

    # Solve for the update vector using the Cholesky decomposition
    L = np.linalg.cholesky(H)
    delta = np.linalg.solve(L.T, np.linalg.solve(L, g))

    # Update the homography matrix
    H += delta.reshape((3, 3))

  return H

In [37]:
# Generate some random points in the two images
# Generate some random points in the two images
X = np.random.rand(10, 3)
Y = np.random.rand(10, 3)

# Compute the homography matrix using the Gauss-Newton method
H = gauss_newton(X, Y)


ValueError: shapes (8,10) and (30,) not aligned: 10 (dim 1) != 30 (dim 0)